### Training Data process

#### 1. load samples

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

WNL = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize(word):
    #
    return WNL.lemmatize(word, get_wordnet_pos(pos_tag([word])[0][1]))

In [ ]:
import json

def load_json(file_path):
    assert file_path.split('.')[-1] == 'json'
    with open(file_path,'r') as file:
        data = json.load(file)
    return data

def save_json(save_path,data):
    assert save_path.split('.')[-1] == 'json'
    with open(save_path, 'w', encoding='utf-8') as file:
        json.dump(data, file)
    file.close()

In [ ]:
def get_word_triple(story, sec_id):
  word_list = {}
  triple_list = []
  story_data = load_json('/gdrive/MyDrive/AI4EDU/Retriever/data/' + story + '.json')
  triples = load_json('/gdrive/MyDrive/AI4EDU/Retriever/triples_train.json')
  for word in story_data[str(int(sec_id)-1)]['words']:
    if word['stop'] == 0:
      word_char = lemmatize(word['word'])
      # print(word_char)
      if word_char not in word_list and word_char in triples:
        word_list[word_char] = ''
        i = 0
        for tp in triples[word_char]['triples']:
          if i == 3:
            break
          word_list[word_char] += '(' + tp[0] + ', ' + tp[1] + ', ' + tp[2] + '); '
          triple_list.append('(' + tp[0] + ', ' + tp[1] + ', ' + tp[2] + ')')
          i += 1
  triple_list_str = ''
  for word in word_list:
    triple_list_str += word_list[word] + '\n'

  return triple_list, triple_list_str

In [ ]:
import csv
import json

training_data = {}
# each line: {"query": str, "pos": List[str], "neg":List[str]}
dir = '/gdrive/MyDrive/AI4EDU/Retriever/train_split.csv'

with open(dir) as file_obj:
    heading = next(file_obj)
    reader_obj = csv.reader(file_obj)
    for row in reader_obj:
      title = row[1]
      sec_id = row[2]
      text = row[3]
      triple_list, triple_list_str = get_word_triple(title, sec_id)
      if title not in training_data:
        training_data[title] = {}
      if sec_id not in training_data[title]:
        training_data[title][sec_id] = {
            "text": text,
            "triple": triple_list_str,
            "pos": [],
            "neg": []
        }
      pos_triple = '(' + row[5] + ', ' + row[6] + ', ' + row[7] + ')'
      neg_triple = []
      for tp in triple_list:
        if tp != pos_triple:
          neg_triple.append(tp)
      if pos_triple not in training_data[title][sec_id]['pos']:
        training_data[title][sec_id]['pos'].append(pos_triple)
        training_data[title][sec_id]['neg'] += neg_triple

retriever_fine_tune_data = []
for title, val in training_data.items():
  for sec_id, val_sec in val.items():
    # retriever_fine_tune_data.append({"query": val_sec['text'] + '\n\n' + val_sec['triple'], "pos": val_sec['pos'], "neg":val_sec['neg']})
    retriever_fine_tune_data.append({"query": val_sec['text'], "pos": val_sec['pos'], "neg":val_sec['neg']})

KeyboardInterrupt: 

In [ ]:
save_json('/gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_data_story_only_full_neg.json', retriever_fine_tune_data)
save_json('/gdrive/MyDrive/AI4EDU/Retriever/all_training_data_story_only_full_neg.json', training_data)

In [ ]:
retriever_fine_tune_data = load_json('/gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_data_story_only_full_neg.json')

with open('/gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_data_story_only_full_neg.jsonl', 'w') as file:
    for item in retriever_fine_tune_data:
        json_str = json.dumps(item)
        file.write(json_str + '\n')

In [ ]:
!pip install evaluate
!pip install rouge_score
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score

In [ ]:
from evaluate import load
from sentence_transformers import SentenceTransformer, util
sent_sim_model = SentenceTransformer('all-MiniLM-L6-v2')

rouge = load('rouge')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 1. BM25

In [ ]:
%pip install langchain
%pip install --upgrade --quiet  rank_bm25
%pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 8.3 MB/s eta 0:00:00


test data

In [ ]:
import csv
TEST_DATA = []

# dir = '/content/drive/MyDrive/AI4EDU/Retriever/test_OneSecPerLine.csv'
dir = '/gdrive/MyDrive/AI4EDU/Retriever/test_OneSecPerLine.csv'



# Open file
with open(dir) as file_obj:
    heading = next(file_obj)
    reader_obj = csv.reader(file_obj)
    for row in reader_obj:
      eachQA = [ [row[3], '(' + row[3] + ', ' + row[4]  + ', ' + row[5] + ')', row[6], row[7]] ]
      if row[11] != '' and row[12] != '':
        eachQA.append( [ row[8], '(' + row[8] + ', ' + row[9]  + ', ' + row[10] + ')', row[11], row[12]] )
      if row[16] != '' and row[17] != '':
        eachQA.append( [row[13], '(' + row[13] + ', ' + row[14]  + ', ' + row[15] + ')', row[16], row[17]] )
      TEST_DATA.append([row[0], row[1], row[2], eachQA])

print(len(TEST_DATA))
print(TEST_DATA[0])

FileNotFoundError: [Errno 2] No such file or directory: '/gdrive/MyDrive/AI4EDU/Retriever/test_OneSecPerLine.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from langchain_community.retrievers import BM25Retriever

all_rouge = all_sentsim = 0

with open('/gdrive/MyDrive/AI4EDU/Retriever/BM25_story_text_finetune.txt', 'w') as txt_file:
  for data in TEST_DATA:
    title = data[0]
    sec_id = data[1]
    text = data[2]
    triple_list, triple_list_str = get_word_triple(title, sec_id)
    retriever = BM25Retriever.from_texts(triple_list)
    pred_triple = retriever.invoke(text)[0].page_content
    txt_file.write(pred_triple + '\n')
    each_rouge = each_sentsim = 0

    embeddings1 = sent_sim_model.encode(pred_triple, convert_to_tensor=True)

    for gt in data[3]:
      gt_triple = gt[1]
      embeddings2 = sent_sim_model.encode(gt_triple, convert_to_tensor=True)
      new_rouge = rouge.compute(predictions=[pred_triple], references=[gt_triple])['rougeL']
      if new_rouge > each_rouge:
        each_rouge = new_rouge


      cosine_scores = util.cos_sim(embeddings1, embeddings2)[0][0].item()

      if cosine_scores > each_sentsim:
        each_sentsim = cosine_scores

    all_rouge += each_rouge
    all_sentsim += each_sentsim

print("rouge: ", all_rouge / len(TEST_DATA))
print("sentsim: ", all_sentsim / len(TEST_DATA))

rouge:  0.4207627988449913
sentsim:  0.3384097928666089


### BGE

In [ ]:
%pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.6 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.2.10-py3-none-any.whl size=166100 sha256=433e03ea71b5d92ff92c7196d95ce36692b18ac632e1271b44d5666eb8cc15f1
  Stored in directory: /root/.cache/pip/wheels/3b/1d/d2/eec38cd59144f4c9767d7c55cfae8e8feec699071aa41ca5da
Successfully built FlagEmbedding


Hard Neg

In [ ]:
%pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00


In [ ]:
!python -m FlagEmbedding.baai_general_embedding.finetune.hn_mine \
--model_name_or_path BAAI/bge-base-en-v1.5 \
--input_file /gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_data_story_only_full_neg.jsonl \
--output_file /gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_data_story_only_full_neg_minedHN.jsonl \
--range_for_sampling 2-200 \
--negative_number 15 \
--use_gpu_for_searching

2024-05-29 15:04:45.513395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 15:04:45.513448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 15:04:45.515157: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 15:04:47.162574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
tokenizer_config.json: 100% 366/366 [00:00<00:00, 2.26MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 3.55MB/s]
tokenizer.json: 100% 711k/711k [00:00<00:00, 3.17MB/s]
special_tokens_map.json: 100% 125/125 [00:00<00:00, 1.05MB/s]
config.json: 1

In [ ]:
!nvidia-smi

Mon May 27 15:34:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!torchrun --nproc_per_node 1 \
-m FlagEmbedding.baai_general_embedding.finetune.run \
--output_dir /gdrive/MyDrive/AI4EDU/Retriever/models/bge-large-en-v1.5-story_only_3 \
--model_name_or_path BAAI/bge-large-en-v1.5 \
--train_data /gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_data_story_only_full_neg_minedHN.jsonl \
--learning_rate 1e-5 \
--fp16 \
--num_train_epochs 5 \
--per_device_train_batch_size 50 \
--dataloader_drop_last True \
--normlized True \
--temperature 0.02 \
--query_max_len 64 \
--passage_max_len 256 \
--train_group_size 2 \
--negatives_cross_device \
--logging_steps 10 \
--save_steps 1000 \
--query_instruction_for_retrieval ""

2024-05-30 14:21:41.134234: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 14:21:41.134284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 14:21:41.139694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 14:21:42.915215: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/30/2024 14:21:47 - WARNING - __main__ -   Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: True
05/30/2024 14:21:47 - INFO - __main__ -   Training/evaluation parameters RetrieverTrainingArguments(
_

#### Construct Eval Data

1. Corpus Data

In [ ]:
with open('/gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_data_full_neg.jsonl', 'r') as r_file:
  with open('/gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_corpus.jsonl', 'w') as w_file:
    for item in r_file:
      item = json.loads(item)
      corpus = {"content": item["pos"][0]}
      # print(corpus)
      json_str = json.dumps(corpus)
      w_file.write(json_str + '\n')


{'content': '(fish, is at location of, water)'}
{'content': '(tribe, is a, social_group)'}
{'content': '(wind, is made of, air)'}
{'content': '(dive, has subevent, hold_breath)'}
{'content': '(prank, is a, joke)'}
{'content': '(hoist, is used for, lift_things)'}
{'content': '(dressmaker, is capable of, cut_cloth)'}
{'content': '(avenue, is a, street)'}
{'content': '(whisper, is a, speaking)'}
{'content': '(prince, is a, son_of_king)'}
{'content': '(wood, is used for, fence_in_property)'}
{'content': '(cloud, is used for, rain)'}
{'content': '(bed, is used for, lie_down)'}
{'content': '(ill, causes, go_to_doctor)'}
{'content': '(affection, is a, good_will)'}
{'content': '(flatten, has subevent, change_shape)'}
{'content': '(bounce, causes, jump_up_and_down)'}
{'content': '(scrape, has subevent, roll_up)'}
{'content': '(firkin, is a, british_capacity_unit)'}
{'content': '(whistle, is a, high_pitched_sound)'}
{'content': '(passion, is a, emotion)'}
{'content': '(succeed, has subevent, wor

In [ ]:
import csv
import json

eval_data = {}
# each line: {"query": str, "pos": List[str], "neg":List[str]}
dir = '/gdrive/MyDrive/AI4EDU/Retriever/val_OneSecPerLine.csv'

with open(dir) as file_obj:
    heading = next(file_obj)
    reader_obj = csv.reader(file_obj)
    for row in reader_obj:
      title = row[0]
      sec_id = row[1]
      text = row[2]
      triple_list, triple_list_str = get_word_triple(title, sec_id)
      if title not in eval_data:
        eval_data[title] = {}
      if sec_id not in eval_data[title]:
        eval_data[title][sec_id] = {
            "text": text,
            "triple": triple_list_str,
            "pos": []
        }
      pos_triple = ['(' + row[3] + ', ' + row[4] + ', ' + row[5] + ')']
      if row[11] != '' and row[12] != '':
        pos_triple.append('(' + row[8] + ', ' + row[9]  + ', ' + row[10] + ')')
      if row[16] != '' and row[17] != '':
        pos_triple.append('(' + row[13] + ', ' + row[14]  + ', ' + row[15] + ')')

      eval_data[title][sec_id]['pos'] += pos_triple

retriever_eval_data = []
for title, val in eval_data.items():
  for sec_id, val_sec in val.items():
    retriever_eval_data.append({"query": val_sec['text'] + '\n\n' + val_sec['triple'], "positive": val_sec['pos']})

In [ ]:
with open('/gdrive/MyDrive/AI4EDU/Retriever/retriever_eval_data.jsonl', 'w') as file:
    for item in retriever_eval_data:
        json_str = json.dumps(item)
        file.write(json_str + '\n')

In [ ]:
!python -m FlagEmbedding.baai_general_embedding.finetune.eval_msmarco \
--encoder /gdrive/MyDrive/AI4EDU/Retriever/models/bge-large-en-v1.5-story_text_only \
--fp16 \
--add_instruction 0 \
--k 100 \
--corpus_data /gdrive/MyDrive/AI4EDU/Retriever/retriever_fine_tune_corpus.jsonl \
--query_data /gdrive/MyDrive/AI4EDU/Retriever/retriever_eval_data.jsonl

2024-05-29 15:48:40.104235: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 15:48:40.104299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 15:48:40.105604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 15:48:41.528293: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Generating train split: 380 examples [00:00, 966.82 examples/s]
Generating train split: 3350 examples [00:00, 612213.78 examples/s]
Inference Embeddings: 100% 14/14 [00:03<00:00,  4.60it/s]
Inference Embeddings: 100% 2/2 [00:01<00:00,  1.24i

Inference

In [ ]:
from evaluate import load
from sentence_transformers import SentenceTransformer, util
from FlagEmbedding import FlagModel

sent_sim_model = SentenceTransformer('all-MiniLM-L6-v2')

rouge = load('rouge')

# model = FlagModel('/gdrive/MyDrive/AI4EDU/Retriever/models/bge-large-en-v1.5-story_only_3',
#                   query_instruction_for_retrieval="",
#                   use_fp16=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from evaluate import load
from sentence_transformers import SentenceTransformer, util
from FlagEmbedding import FlagModel

sent_sim_model = SentenceTransformer('all-MiniLM-L6-v2')

rouge = load('rouge')

model = FlagModel('BAAI/bge-large-en-v1.5',
                  query_instruction_for_retrieval="",
                  use_fp16=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
import csv
import json
import numpy as np

dir = '/gdrive/MyDrive/AI4EDU/Retriever/test_OneSecPerLine.csv'

all_rouge = all_sentsim = 0

with open('/gdrive/MyDrive/AI4EDU/Retriever/BGE_not-fine-tune_.txt', 'w') as txt_file:
  with open(dir) as file_obj:
      heading = next(file_obj)
      reader_obj = csv.reader(file_obj)
      length = 0
      for row in reader_obj:
        length += 1
        title = row[0]
        sec_id = row[1]
        text = row[2]
        triple_list, triple_list_str = get_word_triple(title, sec_id)

        each_rouge = each_sentsim = 0

        pos_triple = ['(' + row[3] + ', ' + row[4] + ', ' + row[5] + ')']
        if row[11] != '' and row[12] != '':
          pos_triple.append('(' + row[8] + ', ' + row[9]  + ', ' + row[10] + ')')
        if row[16] != '' and row[17] != '':
          pos_triple.append('(' + row[13] + ', ' + row[14]  + ', ' + row[15] + ')')

        embeddings_1 = model.encode([text])
        embeddings_2 = model.encode(triple_list)
        similarity = embeddings_1 @ embeddings_2.T
        max_index = np.argmax(similarity)
        matched_triple = triple_list[max_index]
        txt_file.write(matched_triple + '\n')

        sent_embeddings1 = sent_sim_model.encode(matched_triple, convert_to_tensor=True)
        for tp in pos_triple:
          new_rouge = rouge.compute(predictions=[matched_triple], references=[tp])['rougeL']
          if new_rouge > each_rouge:
            each_rouge = new_rouge

          sent_embeddings2 =sent_sim_model.encode(tp, convert_to_tensor=True)
          cosine_scores = util.cos_sim(sent_embeddings1, sent_embeddings2)[0][0].item()

          if cosine_scores > each_sentsim:
            each_sentsim = cosine_scores

        all_rouge += each_rouge
        all_sentsim += each_sentsim


print("rouge: ", all_rouge / length)
print("sentsim: ", all_sentsim / length)

rouge:  0.29846194900989514
sentsim:  0.39525544620948294


In [ ]:
from FlagEmbedding import FlagModel
sentences_1 = ["butterfly is an animal"]
sentences_2 = ["(creature, is a, animal)", "(gay, is a, slang_term_for_homosexual)"]
model = FlagModel('/gdrive/MyDrive/AI4EDU/Retriever/models/bge-large-en-v1.5-1',
                  query_instruction_for_retrieval="retrieve the most educational-appropriate triple for this story text",
                  use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
embeddings_1 = model.encode(sentences_1)
embeddings_2 = model.encode(sentences_2)
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

[[0.684  0.5977]]
